In [1]:
import json 
import pandas as pd
import numpy as np 
import gdown
from langdetect import detect 
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

In [2]:
# VADER es un analizador de sentimientos diseñado específicamente para textos sociales
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to C:\Users\Mati
[nltk_data]     Barna\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [3]:
# SE CARGAN LOS ARCHIVOS ORIGINALES
# Nombre del archivo JSON
nombre_archivo = 'australian_user_reviews.json'

# Se abre el archivo en modo lectura ('r') con codificación utf-8
with open(nombre_archivo, 'r', encoding='utf-8') as archivo:
    # Leer todas las líneas del archivo y almacenarlas en la lista 'datos'
    datos = archivo.readlines()

# Se convierten las líneas en registros utilizando eval y eliminando los espacios alrededor
registros = [eval(line.strip()) for line in datos]

# Se crea un DataFrame de pandas a partir de los registros
df_UserReviews = pd.DataFrame(registros)

In [4]:
# SE EXPLORAN Y ENTIENDEN LOS DATOS
df_UserReviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [5]:
# Se Carga el archivo JSON reparado en un DataFrame
nombre_archivo = 'australian_user_reviews_repair.json'
with open(nombre_archivo, 'r', encoding='utf-8') as archivo:
    DatosNuevos = json.load(archivo)

# Se normalizan los datos y crea un DataFrame
df = pd.json_normalize(DatosNuevos)
# Se crea un nuevo DataFrame (df_UserReviews) con detalles de cada revisión, manteniendo las columnas 'user_id' y 'user_url'.
df_UserReviews = pd.json_normalize(DatosNuevos, record_path=['reviews'], meta=['user_id', 'user_url'])

# Se hace una lista con el orden deseado de las columnas
column_order = ['user_id', 'user_url', 'funny', 'posted', 'last_edited', 'item_id', 'helpful', 'recommend', 'review']

# Se reordena las columnas del DataFrame
df_UserReviews = df_UserReviews[column_order]

df_UserReviews.head()


,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


In [6]:
# Se muestra un resumen de la información del DataFrame df_UserReviews
df_UserReviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      59305 non-null  object
 1   user_url     59305 non-null  object
 2   funny        59305 non-null  object
 3   posted       59305 non-null  object
 4   last_edited  59305 non-null  object
 5   item_id      59305 non-null  object
 6   helpful      59305 non-null  object
 7   recommend    59305 non-null  bool  
 8   review       59305 non-null  object
dtypes: bool(1), object(8)
memory usage: 3.7+ MB


In [7]:
#  SE LIMPIAN Y PREPROCESAN LOS DATOS
# Se verifica cuántos valores nulos hay por columnas
valores_nulos_por_columna = df_UserReviews.isnull().sum()

# Se muestran los resultados
print(valores_nulos_por_columna)

# Se reemplazan valores vacíos, 'null' y 'None' con NaN en todo el DataFrame
df_UserReviews.replace(['', 'null', 'None'], np.nan, inplace=True)

user_id        0
user_url       0
funny          0
posted         0
last_edited    0
item_id        0
helpful        0
recommend      0
review         0
dtype: int64


In [8]:
# Se verifica cuántos valores nulos hay por columnas
valores_nulos_por_columna = df_UserReviews.isnull().sum()

# Se muestra los resultados
print(valores_nulos_por_columna)

user_id            0
user_url           0
funny          51154
posted             0
last_edited    53165
item_id            0
helpful            0
recommend          0
review            30
dtype: int64


In [9]:
# Filas donde todas las columnas especificadas tienen valores nulos.
columnas_a_considerar = ['posted', 'recommend', 'review']
filas_con_nulos = df_UserReviews[df_UserReviews[columnas_a_considerar].isnull().all(axis=1)]
filas_con_nulos

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review


In [10]:
# Podemos ver que funny y last_edited tienen la mayoria de los datos nulos, por ende se
# se procedera a eliminar esas columnas al igual que helpfull y user_url ya que no aportan
# datos importantes a nuestro estudio.

# Se eliminan las columnas 'helpful' y 'user_url' por considerarse no relevantes, y las columnas
# 'funny' y 'last_edited' por tener la mayoria de datos faltantes
df_UserReviews.drop(['helpful','user_url','funny','last_edited'], axis=1, inplace=True)

In [11]:
# Se buscan registros duplicados
df_UserReviews.sort_values('user_id')
duplicados = df_UserReviews[df_UserReviews.duplicated(subset=['user_id', 'item_id', 'posted', 'review'], keep=False)]
duplicados.count()


user_id      1734
posted       1734
item_id      1734
recommend    1734
review       1734
dtype: int64

In [12]:

# Se eliminan los registros duplicados basándome en múltiples columnas 
df_UserReviews.drop_duplicates(subset=['user_id', 'item_id', 'posted', 'review'], keep=False, inplace=True)

cantidad_total_duplicados_eliminados = duplicados.shape[0]

print("Cantidad total de registros duplicados eliminados:", cantidad_total_duplicados_eliminados)

Cantidad total de registros duplicados eliminados: 1734


In [13]:
# Se convierte 'posted' a tipo datetime
df_UserReviews['posted'] = pd.to_datetime(df_UserReviews['posted'].astype(str).str.replace(r'Posted |,|\.', '', regex=True), errors='coerce')

# Se crea la columna 'year' a partir de 'posted'
df_UserReviews['year'] = df_UserReviews['posted'].dt.year.astype('Int64')

# Se ordena el DataFrame por 'item_id' y 'year' para asegurar que la interpolación se haga correctamente
df_UserReviews = df_UserReviews.sort_values(['item_id', 'year'])

# Se imprime información sobre nulos en 'year' después de la conversión
print("Nulos después de la conversión a datetime:")
print(df_UserReviews['year'].isnull().sum())

Nulos después de la conversión a datetime:
9755


In [14]:
tipo_dato_posted = df_UserReviews['posted'].dtype

print("Tipo de dato de 'posted':", tipo_dato_posted)

Tipo de dato de 'posted': datetime64[ns]


In [15]:
# Rellenamos valores nulos en 'year' mediante interpolación lineal por grupo (item_id)
df_UserReviews['year'] = df_UserReviews.groupby('item_id', group_keys=False)['year'].apply(lambda group: group.interpolate(method='pad') if group.notna().any() else group)

# Verificamos si hay valores nulos en la columna 'year'
hay_nulos_en_year = df_UserReviews['year'].isnull().any()

if hay_nulos_en_year:
    print("Hay valores nulos en la columna 'year'.")
else:
    print("No hay valores nulos en la columna 'year'.")


C:\Users\Mati Barna\AppData\Local\Temp\ipykernel_10364\496998528.py:2: FutureWarning: Series.interpolate with method=pad is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_UserReviews['year'] = df_UserReviews.groupby('item_id', group_keys=False)['year'].apply(lambda group: group.interpolate(method='pad') if group.notna().any() else group)


Hay valores nulos en la columna 'year'.


In [16]:
# Aún hay valores nulos después de la interpolación, se llenan con la mediana. 
df_UserReviews['year'] = df_UserReviews['year'].fillna(df_UserReviews['year'].median())

In [17]:
# Volvemos a verificar si hay valores nulos en la columna 'year'
hay_nulos_en_year = df_UserReviews['year'].isnull().any()

if hay_nulos_en_year:
    print("Hay valores nulos en la columna 'year'.")
else:
    print("No hay valores nulos en la columna 'year'.")

No hay valores nulos en la columna 'year'.


In [18]:
# Se eliminan las columnas 'posted' y 'user_id' que ya no son de utilidad
df_UserReviews.drop(['posted', 'user_id'], axis=1, inplace=True)
df_UserReviews.head()

,item_id,recommend,review,year
5331,10,True,this game is the 1# online action game is awes...,2011
22702,10,True,GYERTEK GAMELNI MINDENKI ITT VAN AKI SZÁMIT !!...,2012
35539,10,True,:D,2012
43134,10,True,Good Game :D,2012
24137,10,True,jueguenlooooooo,2013


In [19]:
# Análisis de texto para determinar el idioma de las reseñas
def detectar_idioma(texto):
    try:
       return detect(texto)
    except:
        return None

# Se crea una nueva columna 'idioma', aplicando la funcion a la columna 'review'    
df_UserReviews['language']=df_UserReviews['review'].apply(detectar_idioma)

# Se calcula el conteo y porcentaje de cada idioma
conteo_por_idioma = df_UserReviews['language'].value_counts()
porcentaje_por_idioma = df_UserReviews['language'].value_counts(normalize=True) * 100

# Se crea un nuevo DataFrame con el conteo y porcentaje
conteo_porcentaje_idiomas = pd.DataFrame({
    'Conteo': conteo_por_idioma,
    'Porcentaje': porcentaje_por_idioma.round(2).astype(str) + '%'
})

# Se ordena el dataframe por el conteo de mayor a menor  
conteo_porcentaje_idiomas=conteo_porcentaje_idiomas.sort_values(by='Conteo',ascending=False)
conteo_porcentaje_idiomas.head()



,Conteo,Porcentaje
language,,
en,44409,77.89%
pt,2118,3.71%
es,1245,2.18%
de,1125,1.97%
so,995,1.75%


In [20]:
# Mapeo de códigos de idioma a nombres completos para el análisis de idiomas (top 5)
nombres_completos={
    'en': 'Inglés',
    'pt': 'Portugués',
    'es': 'Español',
    'de': 'Alemán',
    'so': 'Somalí',
}

# Se aplica el mapeo al DataFrame
df_UserReviews['idioma_completo'] = df_UserReviews['language'].map(nombres_completos)

# Se crea un DataFrame con el resumen de idiomas
idiomas_top5 = df_UserReviews['idioma_completo'].value_counts().reset_index()
idiomas_top5.columns = ['Idioma', 'Conteo']
idiomas_top5['Conteo'] = idiomas_top5['Conteo'].round(2)

# Se calcula el porcentaje
idiomas_top5['Porcentaje'] = (idiomas_top5['Conteo'] / len(df_UserReviews)) * 100 
idiomas_top5['Porcentaje'] = idiomas_top5['Porcentaje'].round(2)
idiomas_top5.head()

,Idioma,Conteo,Porcentaje
0,Inglés,44409,77.14
1,Portugués,2118,3.68
2,Español,1245,2.16
3,Alemán,1125,1.95
4,Somalí,995,1.73


In [21]:
# Se selecciona las filas donde el idioma es 'ingles'
df_UserReviews = df_UserReviews.query("idioma_completo == 'Inglés'")

# Se borra la columna 'idioma_completo'
df_UserReviews = df_UserReviews.drop('idioma_completo', axis=1)

In [22]:
# Se convierten todas las letras a minúsculas para asegurar que todas las palabras sean tratadas de la misma manera.
df_UserReviews.loc[:, 'review'] = df_UserReviews['review'].str.lower()

# Eliminación de caracteres especiales(excepto Alfanuméricos y Espacios)
df_UserReviews.loc[:,'review'] = df_UserReviews['review'].replace('[^A-Za-z0-9\s]+', '', regex=True)

# Se elimina caracteres de puntuación que no aportan al análisis de sentimiento.
df_UserReviews.loc[:, 'review'] = df_UserReviews['review'].str.replace('[^\w\s]', '', regex=True)

In [23]:
df_UserReviews.head()

,item_id,recommend,review,year,language
5331,10,True,this game is the 1 online action game is aweso...,2011,en
45506,10,True,the og to csgo,2013,en
7801,10,True,the best fps game,2014,en
7967,10,True,one of the best childhood games i have ever pl...,2014,en
8519,10,True,people still play this siq game,2014,en


In [24]:
def analyze_sentiments(df):
    
    # Se instancia el analizador de sentimientos
    sia = SentimentIntensityAnalyzer()

    # Se aplica el análisis de sentimientos y asignar valores numéricos
    df['compound_score'] = df['review'].apply(lambda review: sia.polarity_scores(review)['compound'])
    df['sentiment_analysis'] = df['compound_score'].apply(lambda score: 0 if score < 0 else (1 if score == 0 else 2))

    # Conteo de reviews por score
    conteo_por_score = df['sentiment_analysis'].value_counts()

    # Conteo de reviews en blanco
    reviews_en_blanco = df['review'].isnull().sum()

    # Total de reviews
    total_reviews = len(df)

    # Se calculan porcentajes
    porcentaje_score = (conteo_por_score / total_reviews * 100).round(2)
    porcentaje_reviews_vacias = (reviews_en_blanco / total_reviews * 100).round(2)
     
    # Se eliminan las columnas 'review' y 'compound_score', no necesitaremos estos datos
    df.drop(['compound_score'], axis=1, inplace=True)  

    return df, conteo_por_score, reviews_en_blanco, porcentaje_score, porcentaje_reviews_vacias



In [25]:
# Se llama a la función analyze_sentiments
df_UserReviews, conteo_por_score, reviews_en_blanco, porcentaje_score, porcentaje_reviews_vacias = analyze_sentiments(df_UserReviews)

# Se crea un nuevo DataFrame con el conteo y porcentaje
sentimientos = pd.DataFrame({
    'Conteo': conteo_por_score,
    'Porcentaje': porcentaje_score.round(2).astype(str) + '%'
})

# Se ordena el DataFrame por el conteo de mayor a menor
sentimientos = sentimientos.sort_values(by='Conteo', ascending=False)

# Se imprimen los resultados
print("\nAnálisis de sentimientos:")
print(sentimientos)
print("\nConteo de reviews en blanco: ", reviews_en_blanco, " Porcentaje: ", porcentaje_reviews_vacias.round(2).astype(str) + '%')



Análisis de sentimientos:
                    Conteo Porcentaje
sentiment_analysis                   
2                    31461     70.84%
0                     8273     18.63%
1                     4675     10.53%

Conteo de reviews en blanco:  0  Porcentaje:  0.0%


In [26]:
# Se borra la columna 'language'
df_UserReviews = df_UserReviews.drop('language', axis=1)
df_UserReviews

,item_id,recommend,review,year,sentiment_analysis
5331,10,True,this game is the 1 online action game is aweso...,2011,2
45506,10,True,the og to csgo,2013,1
7801,10,True,the best fps game,2014,2
7967,10,True,one of the best childhood games i have ever pl...,2014,2
8519,10,True,people still play this siq game,2014,2
...,...,...,...,...,...
51725,99900,True,a fun free to play game at first but eventuall...,2015,2
53065,99900,False,elite orbs are more rarer than good presidenti...,2015,2
12393,99910,True,looks like a cute childrens game but has more ...,2011,2
53052,99910,False,110 just a bad game overall dead boring dead s...,2014,0


In [27]:
# SE GUARDA EL ARCHIVO LIMPIO

# El archivos se almacenan en local 
df_UserReviews.to_csv('user_reviews_cleaned.csv', index=False)

In [28]:
df_UserReviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44409 entries, 5331 to 32632
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   item_id             44409 non-null  object
 1   recommend           44409 non-null  bool  
 2   review              44409 non-null  object
 3   year                44409 non-null  Int64 
 4   sentiment_analysis  44409 non-null  int64 
dtypes: Int64(1), bool(1), int64(1), object(2)
memory usage: 1.8+ MB
